In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import matplotlib.cm as cmx
import matplotlib.colors as colors

## Cubic spline

In [2]:
def poly_solver(x0,xt,xw,t0,t1,tw):
    x = np.array([[1,t0,t0**2,t0**3],[1,t1,t1**2,t1**3],[1,tw,tw**2,tw**3],[0,1,2*t0,3*t0**2],[0,1,2*t1,3*t1**2],[0,1,2*tw,3*tw**2]])
    b = np.array([[x0],[xt],[xw],[0],[0],[5]])
    a = np.linalg.lstsq(x,b)[0]
    return a

In [3]:
def gib_spline(n,x0,xt,xw,t0,t1,tw):
    a = poly_solver(x0,xt,xw,t0,t1,tw)
    tlist = np.linspace(t0,t1,n)
#     tlist = np.linspace(0,20,500)
    xlist = [(a[0]+a[1]*i+a[2]*i**2+a[3]*i**3) for i in tlist]
    return tlist,xlist

## Bernstein polynomial (degree=3)

In [7]:
from scipy.interpolate import BPoly
from scipy.special import comb

def bern_coeff_solver(x0,xt,xw,t0,t1,tw):
    x = np.array([[(1-t0)**3,3*t0*(1-t0)**2,3*t0**2*(1-t0),t0**3],[(1-t1)**3,3*t1*(1-t1)**2,3*t1**2*(1-t1),t1**3],[(1-tw)**3,3*tw*(1-tw)**2,3*tw**2*(1-tw),tw**3],[-3*(1-t0)**2,3-12*t0+9*t0**2,6*t0-9*t0**2,3*t0**2],[-3*(1-t1)**2,3-12*t1+9*t1**2,6*t1-9*t1**2,3*t1**2],[-3*(1-tw)**2,3-12*tw+9*tw**2,6*tw-9*tw**2,3*tw**2]])
    b = np.array([[x0],[xt],[xw],[0],[0],[5]])
    a = np.linalg.lstsq(x,b)[0]
    return a

def gib_bernie(n,x0,xt,xw,t0,t1,tw):
    a = bern_coeff_solver(x0,xt,xw,t0,t1,tw)
    tlist = np.linspace(t0,t1,n)
    xlist = [(a[0]*(1-i)**3+a[1]*3*i*(1-i)**2+a[2]*3*i**2*(1-i)+a[3]*i**3) for i in tlist]
    return tlist,xlist

In [8]:
time_samples = 200

class agent:
    def __init__(self,rad,tf,tw,start,waypt,goal):
        self.rad = rad
        self.t0 = 0
        self.tf = tf
        self.tw = tw
        self.pos = start
        self.locations_x = []
        self.locations_y = []
        self.goal = goal
        self.origin = start
        self.waypt = waypt
        
    
    def get_traj_x(self):
        times,traj_x = gib_bernie(time_samples,self.origin[0],self.goal[0],self.waypt[0],self.t0,self.tf,self.tw)
        return times,traj_x
    
    def get_traj_y(self):
        times,traj_y = gib_bernie(time_samples,self.origin[1],self.goal[1],self.waypt[1],self.t0,self.tf,self.tw)
        return traj_y
            
    def get_overall_traj(self):
        times,traj_x = self.get_traj_x()
        traj_y = self.get_traj_y()
        for i in range(len(times)):
            self.pos = [traj_x[i],traj_y[i]]
            self.locations_x.append(self.pos[0])
            self.locations_y.append(self.pos[1])
            self.visualize_traj(i)
              
    def visualize_traj(self,time):
        figure = plt.figure()
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Circle(
            (self.pos[0],self.pos[1]),
            radius = self.rad,
            edgecolor='black',
            linewidth=1.0,
            ls='solid',
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        name = 'data_poly/snap%s.png'%str(time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="red", markersize =15,linewidth=3.0)
        ax.plot(self.locations_x,self.locations_y,"b--")
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 60.0)
        ax.set_ylim(-10.0, 60.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure

In [9]:
bot = agent(2,20,5,[0,0],[5,40],[40,40])
bot.get_overall_traj()

/home/susiejojo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  import sys
